In [3]:
from pyspark.sql import SparkSession, functions as F, types as T
from pyspark import SparkConf
from dotenv import load_dotenv
import os

In [4]:
conf = SparkConf()
conf.set("spark.jars.packages", "net.snowflake:snowflake-jdbc:3.24.2,net.snowflake:spark-snowflake_2.12:3.1.2") 
spark = SparkSession.builder \
    .config(conf=conf) \
    .getOrCreate() 

In [ ]:
load_dotenv(dotenv_path="/home/jovyan/work/.env")

sfOptions = { 
    "sfURL" : os.getenv("URL"),
    "sfDatabase" : os.getenv("DB"),
    "sfSchema" : os.getenv("SCHEMA"),
    "sfWarehouse" :os.getenv("WH"),
    "sfRole" : os.getenv("ROLE"),
    "sfUser" : os.getenv("USER"),
    "sfPassword" : os.getenv("PASSWORD")
}


df = spark.read \
    .format("snowflake") \
    .options(**sfOptions) \
    .option("dbtable", "STAGE.OBT_TRIPS") \
    .load()

a. Top 10 zonas de pickup por volumen mensual 

In [4]:
top_pickup = (
    df.groupBy("MONTH", "PU_ZONE")
      .agg(F.count("*").alias("num_trips"))
      .orderBy( "MONTH", F.col("num_trips").desc())
)
top_pickup.show()

+-----+--------------------+---------+
|MONTH|             PU_ZONE|num_trips|
+-----+--------------------+---------+
|    1|Upper East Side S...|  2883879|
|    1|Upper East Side N...|  2773822|
|    1|      Midtown Center|  2720475|
|    1|        Midtown East|  2363997|
|    1|Times Sq/Theatre ...|  2351888|
|    1|Penn Station/Madi...|  2320950|
|    1|            Union Sq|  2173714|
|    1|         Murray Hill|  2146120|
|    1|        Clinton East|  2081269|
|    1| Lincoln Square East|  2060589|
|    1|        East Village|  1991352|
|    1|         JFK Airport|  1909270|
|    1|       Midtown North|  1852674|
|    1|Upper West Side S...|  1829820|
|    1|     Lenox Hill West|  1712673|
|    1|        East Chelsea|  1656011|
|    1|            Gramercy|  1654541|
|    1|   LaGuardia Airport|  1618666|
|    1|       Midtown South|  1584655|
|    1|        West Village|  1467908|
+-----+--------------------+---------+
only showing top 20 rows



Esta tabla recopila el orden por volumen de cada mes y su correspondiente zona. En este caso podemos ver que de el mes de enero(global) tiene dentro de su top 10: Upper East Side, hasta Lincoln Square East

b. Top 10 zonas de dropoff por volumen mensual 

In [5]:
top_dropoff = (
    df.groupBy("MONTH", "DO_ZONE")
      .agg(F.count("*").alias("num_trips"))
      .orderBy( "MONTH", F.col("num_trips").desc())
)
top_dropoff.show()

+-----+--------------------+---------+
|MONTH|             DO_ZONE|num_trips|
+-----+--------------------+---------+
|    1|Upper East Side N...|  2933573|
|    1|Upper East Side S...|  2597709|
|    1|      Midtown Center|  2590444|
|    1|         Murray Hill|  2178945|
|    1|Times Sq/Theatre ...|  2092417|
|    1|        Midtown East|  2060039|
|    1| Lincoln Square East|  1902618|
|    1|            Union Sq|  1896976|
|    1|        Clinton East|  1871501|
|    1|Upper West Side S...|  1836622|
|    1|     Lenox Hill West|  1792438|
|    1|Penn Station/Madi...|  1768166|
|    1|        East Village|  1719326|
|    1|       Midtown North|  1651736|
|    1|        East Chelsea|  1594994|
|    1|Upper West Side N...|  1582644|
|    1|            Gramercy|  1501990|
|    1|      Yorkville West|  1438814|
|    1|       Midtown South|  1426708|
|    1|     Lenox Hill East|  1377503|
+-----+--------------------+---------+
only showing top 20 rows



Esta tabla recopila el orden por volumen de cada mes y su correspondiente zona. En este caso podemos ver que de el mes de enero(global) tiene dentro de su top 10: Upper East Side, hasta Uper West Side (en el caso de dropoff, el top cambia con respecto a la pregunta anterior)

c. Evolución mensual de total_amount y tip_pct por borough

In [6]:
evolucion_b_t = (
    df.groupBy("MONTH", "PU_BOROUGH")
      .agg(
          F.round(F.avg("TOTAL_AMOUNT"), 2).alias("avg_total_amount"),
          F.round(F.avg("TIP_PCT"), 2).alias("avg_tip_pct")
      )
      .orderBy("PU_BOROUGH", "MONTH")
)
evolucion_b_t.show(50, False)

+-----+----------+----------------+-----------+
|MONTH|PU_BOROUGH|avg_total_amount|avg_tip_pct|
+-----+----------+----------------+-----------+
|1    |Bronx     |18.61           |2.16       |
|2    |Bronx     |19.14           |1.76       |
|3    |Bronx     |18.96           |1.82       |
|4    |Bronx     |19.72           |2.26       |
|5    |Bronx     |19.98           |2.34       |
|6    |Bronx     |20.47           |2.47       |
|7    |Bronx     |21.09           |2.56       |
|8    |Bronx     |21.02           |2.76       |
|9    |Bronx     |20.94           |2.85       |
|10   |Bronx     |21.43           |2.8        |
|11   |Bronx     |21.25           |2.68       |
|12   |Bronx     |21.32           |2.5        |
|1    |Brooklyn  |17.09           |8.39       |
|2    |Brooklyn  |17.74           |8.27       |
|3    |Brooklyn  |17.94           |8.18       |
|4    |Brooklyn  |18.15           |8.38       |
|5    |Brooklyn  |18.54           |8.37       |
|6    |Brooklyn  |18.77           |8.25 

d. Ticket promedio por service_type y mes

In [7]:
ticket_service = (
    df.groupBy("MONTH", "SERVICE_TYPE")
      .agg(F.round(F.avg("TOTAL_AMOUNT"), 2).alias("avg_total_amount"))
      .orderBy("MONTH")
)
ticket_service.show()

+-----+------------+----------------+
|MONTH|SERVICE_TYPE|avg_total_amount|
+-----+------------+----------------+
|    1|      yellow|           17.43|
|    1|       green|           15.25|
|    2|       green|           15.29|
|    2|      yellow|           18.25|
|    3|      yellow|           18.45|
|    3|       green|           15.32|
|    4|      yellow|           18.65|
|    4|       green|           15.56|
|    5|      yellow|           18.88|
|    5|       green|           15.93|
|    6|      yellow|           18.94|
|    6|       green|            15.9|
|    7|      yellow|           19.01|
|    7|       green|           15.86|
|    8|       green|           15.97|
|    8|      yellow|           18.52|
|    9|      yellow|           19.37|
|    9|       green|           16.29|
|   10|      yellow|           19.27|
|   10|       green|           16.12|
+-----+------------+----------------+
only showing top 20 rows



En general el avg total_amount de yellow service es mayor al de green por mes. Esto tiene sentido pues la cantidad de viajes por servicio es significativamente mayor para yellow

e. Viajes por hora del día y día de semana (picos)

In [8]:
trips_per_hour_day = (
    df.groupBy("PICKUP_HOUR", "DAY_OF_WEEK")
      .agg(F.count("*").alias("num_trips"))
      .orderBy("DAY_OF_WEEK", "PICKUP_HOUR")
)
trips_per_hour_day.show(200)

+-----------+-----------+---------+
|PICKUP_HOUR|DAY_OF_WEEK|num_trips|
+-----------+-----------+---------+
|          0|          0|  1758576|
|          1|          0|  1077082|
|          2|          0|  1236917|
|          3|          0|  1804737|
|          4|          0|  2660212|
|          5|          0|  3805086|
|          6|          0|  4813043|
|          7|          0|  5473112|
|          8|          0|  5832409|
|          9|          0|  5891671|
|         10|          0|  5957488|
|         11|          0|  5800303|
|         12|          0|  5774197|
|         13|          0|  5910879|
|         14|          0|  5742830|
|         15|          0|  5220284|
|         16|          0|  4783419|
|         17|          0|  4496293|
|         18|          0|  3824576|
|         19|          0|  2936485|
|         20|          0|  1901274|
|         21|          0|  1156584|
|         22|          0|   763146|
|         23|          0|   616612|
|          0|          1|   

f. p50/p90 de trip_duration_min por borough de pickup

In [6]:
duration_p = (
    df.groupBy("PU_BOROUGH")
      .agg(
          F.percentile_approx("TRIP_DURATION_MIN", 0.5).alias("p50_duration"),
          F.percentile_approx("TRIP_DURATION_MIN", 0.9).alias("p90_duration")
      )
)
duration_p.show()

+-------------+------------+------------+
|   PU_BOROUGH|p50_duration|p90_duration|
+-------------+------------+------------+
|       Queens|        24.0|        54.0|
|          EWR|         0.0|         2.0|
|      Unknown|        10.0|        28.0|
|     Brooklyn|        13.0|        33.0|
|Staten Island|        22.0|        70.0|
|          N/A|         1.0|        60.0|
|    Manhattan|        11.0|        25.0|
|        Bronx|        13.0|        37.0|
+-------------+------------+------------+



Queens y Staten Island mejoran respectivamente tanto en p50 como en p90

g. avg_speed_mph por franja horaria (6-9, 17-20) y borough

In [7]:
df_speed = df.withColumn("time_slot", 
    F.when((F.col("PICKUP_HOUR").between(6,9)), "morning")
   .when((F.col("PICKUP_HOUR").between(17,20)), "evening")
   .otherwise("other")
)

# speed con valores atipicos eliminados
df_speed = df_speed.filter((F.col("AVG_SPEED_MPH") >= 1) & (F.col("AVG_SPEED_MPH") <= 50))

avg_speed_slot = (
    df_speed.groupBy("PU_BOROUGH", "time_slot")
            .agg(F.round(F.avg("AVG_SPEED_MPH"),2).alias("avg_speed_mph"))
            .orderBy("PU_BOROUGH", "time_slot")
)
avg_speed_slot.show(40, False)

+-------------+---------+-------------+
|PU_BOROUGH   |time_slot|avg_speed_mph|
+-------------+---------+-------------+
|Bronx        |evening  |16.43        |
|Bronx        |morning  |13.74        |
|Bronx        |other    |13.96        |
|Brooklyn     |evening  |13.8         |
|Brooklyn     |morning  |11.66        |
|Brooklyn     |other    |12.61        |
|EWR          |evening  |26.14        |
|EWR          |morning  |21.01        |
|EWR          |other    |17.95        |
|Manhattan    |evening  |12.51        |
|Manhattan    |morning  |9.58         |
|Manhattan    |other    |10.87        |
|N/A          |evening  |17.65        |
|N/A          |morning  |13.72        |
|N/A          |other    |14.54        |
|Queens       |evening  |23.59        |
|Queens       |morning  |18.17        |
|Queens       |other    |18.35        |
|Staten Island|evening  |23.05        |
|Staten Island|morning  |20.73        |
|Staten Island|other    |20.95        |
|Unknown      |evening  |13.87        |


Evening lleva el mayor avg_speed_mph en todas las zonas.

h. Participación por payment_type_desc y su relación con tip_pct

In [8]:
payment_tip = (
    df.groupBy("PAYMENT_TYPE_DESC")
      .agg(
          F.count("*").alias("num_trips"),
          F.round(F.avg("TIP_PCT"),2).alias("avg_tip_pct")
      )
      .orderBy(F.col("num_trips").desc())
)
payment_tip.show()

+-----------------+---------+-----------+
|PAYMENT_TYPE_DESC|num_trips|avg_tip_pct|
+-----------------+---------+-----------+
|      Credit card|534452087|      15.05|
|             Cash|241753107|        0.0|
|  Flex Fare trip | 15508659|       17.3|
|        No charge|  3880607|       0.02|
|          Dispute|  3334326|       0.03|
|    Not specified|  1908450|       3.12|
|          Unknown|     3106|        0.1|
+-----------------+---------+-----------+



i. ¿Qué rate_code_desc concentran mayor trip_distance y total_amount?

In [9]:
rate_codes = (
    df.groupBy("RATE_CODE_DESC")
      .agg(
          F.round(F.sum("TRIP_DISTANCE"),2).alias("total_distance"),
          F.round(F.sum("TOTAL_AMOUNT"),2).alias("total_amount"),
          F.count("*").alias("num_trips")
      )
      .orderBy(F.col("total_distance").desc())
)
rate_codes.show(10)

+--------------------+---------------+-----------------+---------+
|      RATE_CODE_DESC| total_distance|     total_amount|num_trips|
+--------------------+---------------+-----------------+---------+
|       Standard rate|3.40412228243E9|1.251075209098E10|756492156|
|             Unknown|  7.207172095E8|   5.1152051399E8| 18581448|
|                 JFK| 4.0683391549E8|  1.29601171011E9| 18437653|
|     Negotiated fare|  4.975935107E7|   2.7371541419E8|  5027895|
|              Newark|  2.654675993E7|   1.4921865334E8|  1627200|
|Nassau or Westche...|  1.994124055E7|    6.423583583E7|   667065|
|          Group ride|        8673.43|        173101.49|     6925|
+--------------------+---------------+-----------------+---------+



j. Mix yellow vs green por mes y borough.

In [10]:
mix_service = (
    df.groupBy("MONTH", "PU_BOROUGH", "SERVICE_TYPE")
      .agg(F.count("*").alias("num_trips"))
      .orderBy("MONTH", "PU_BOROUGH")
)
mix_service.show()

+-----+-------------+------------+---------+
|MONTH|   PU_BOROUGH|SERVICE_TYPE|num_trips|
+-----+-------------+------------+---------+
|    1|        Bronx|      yellow|   107823|
|    1|        Bronx|       green|   431537|
|    1|     Brooklyn|      yellow|   929683|
|    1|     Brooklyn|       green|  2713943|
|    1|          EWR|      yellow|     5235|
|    1|          EWR|       green|      142|
|    1|    Manhattan|      yellow| 62684583|
|    1|    Manhattan|       green|  2373824|
|    1|          N/A|      yellow|    52061|
|    1|          N/A|       green|     3966|
|    1|       Queens|      yellow|  4336075|
|    1|       Queens|       green|  2115805|
|    1|Staten Island|      yellow|     2825|
|    1|Staten Island|       green|     1658|
|    1|      Unknown|      yellow|  1020150|
|    1|      Unknown|       green|    12092|
|    2|        Bronx|      yellow|   104596|
|    2|        Bronx|       green|   377194|
|    2|     Brooklyn|      yellow|   844902|
|    2|   

k. Top 20 flujos PU→DO por volumen y su ticket promedio.

In [11]:
flows = (
    df.groupBy("PU_ZONE", "DO_ZONE")
      .agg(F.count("*").alias("num_trips"),
           F.round(F.avg("TOTAL_AMOUNT"),2).alias("avg_ticket"))
      .orderBy(F.col("num_trips").desc())
)
flows.show(20)

+--------------------+--------------------+---------+----------+
|             PU_ZONE|             DO_ZONE|num_trips|avg_ticket|
+--------------------+--------------------+---------+----------+
|                 N/A|                 N/A|  7078062|     17.83|
|Upper East Side S...|Upper East Side N...|  4145943|     10.34|
|Upper East Side N...|Upper East Side S...|  3549991|     11.25|
|Upper East Side N...|Upper East Side N...|  3298781|      8.57|
|Upper East Side S...|Upper East Side S...|  3157821|      9.11|
|Upper West Side S...|Upper West Side N...|  1856057|      8.92|
|Upper West Side S...| Lincoln Square East|  1839525|      9.45|
|Upper East Side S...|      Midtown Center|  1789514|     12.14|
|Upper East Side S...|        Midtown East|  1783699|     10.73|
| Lincoln Square East|Upper West Side S...|  1753764|      9.92|
|      Midtown Center|Upper East Side S...|  1709279|     11.77|
|Upper West Side N...|Upper West Side S...|  1606176|      8.89|
|     Lenox Hill West|Upp

l. Distribución de passenger_count y efecto en total_amount.

In [12]:
# passenger count debe ser mayor a 0 y menor a 6 y total amount mayor a 0 y menor a 500

df = df.filter((F.col("PASSENGER_COUNT") > 0) & (F.col("PASSENGER_COUNT") < 7) &
                (F.col("TOTAL_AMOUNT") > 0) & (F.col("TOTAL_AMOUNT") < 500))


passenger_effect = (
    df.groupBy("PASSENGER_COUNT")
      .agg(
          F.count("*").alias("num_trips"),
          F.round(F.avg("TOTAL_AMOUNT"),2).alias("avg_total_amount")
      )
      .orderBy("PASSENGER_COUNT")
)
passenger_effect.show()

+---------------+---------+----------------+
|PASSENGER_COUNT|num_trips|avg_total_amount|
+---------------+---------+----------------+
|              1|570072032|           18.15|
|              2|109384461|           19.63|
|              3| 30344967|           19.02|
|              4| 14645485|           20.12|
|              5| 31545128|           17.07|
|              6| 19372282|           16.89|
+---------------+---------+----------------+



m. Impacto de tolls_amount y congestion_surcharge por zona.

In [13]:
fees_effect = (
    df.groupBy("PU_ZONE")
      .agg(
          F.round(F.avg("TOLLS_AMOUNT"),2).alias("avg_tolls"),
          F.round(F.avg("CONGESTION_SURCHARGE"),2).alias("avg_congestion"),
          F.round(F.avg("TOTAL_AMOUNT"),2).alias("avg_total_amount")
      )
      .orderBy(F.col("avg_congestion").desc())
)
fees_effect.show()

+--------------------+---------+--------------+----------------+
|             PU_ZONE|avg_tolls|avg_congestion|avg_total_amount|
+--------------------+---------+--------------+----------------+
|West Chelsea/Huds...|     0.16|          2.95|           16.91|
|      Yorkville East|     0.19|          2.95|           15.66|
|        East Village|     0.08|          2.95|           15.89|
|Upper East Side S...|     0.07|          2.95|           14.14|
| Lincoln Square East|     0.11|          2.95|           15.41|
|        West Village|     0.09|          2.95|           15.91|
|Sutton Place/Turt...|     0.11|          2.95|            15.1|
| Little Italy/NoLiTa|     0.05|          2.95|           16.77|
|Upper West Side S...|     0.13|          2.95|           15.07|
|Greenwich Village...|     0.06|          2.95|           16.16|
|        Battery Park|      0.2|          2.95|            22.9|
|     Lenox Hill West|     0.09|          2.95|           14.36|
|                SoHo|   

n. Proporción de viajes cortos vs largos por borough y estacionalidad.

In [14]:
df_length = df.withColumn(
    "trip_type", 
    F.when((F.col("TRIP_DISTANCE") > 0) & (F.col("TRIP_DISTANCE") <= 5), "short")
     .when(F.col("TRIP_DISTANCE") > 5, "long")
     .otherwise("other") 
)
trip_type_ratio = (
    df_length.groupBy("PU_BOROUGH", "MONTH", "trip_type")
             .agg(F.count("*").alias("num_trips"))
             .orderBy("PU_BOROUGH", "MONTH")
)
trip_type_ratio.show(100)

+----------+-----+---------+---------+
|PU_BOROUGH|MONTH|trip_type|num_trips|
+----------+-----+---------+---------+
|     Bronx|    1|    other|    28564|
|     Bronx|    1|     long|    91523|
|     Bronx|    1|    short|   333612|
|     Bronx|    2|    short|   296278|
|     Bronx|    2|     long|    84610|
|     Bronx|    2|    other|    22308|
|     Bronx|    3|    other|    24301|
|     Bronx|    3|     long|   101109|
|     Bronx|    3|    short|   344355|
|     Bronx|    4|    other|    22063|
|     Bronx|    4|    short|   291392|
|     Bronx|    4|     long|    90534|
|     Bronx|    5|     long|    95238|
|     Bronx|    5|    other|    24048|
|     Bronx|    5|    short|   298300|
|     Bronx|    6|    short|   272747|
|     Bronx|    6|     long|    89791|
|     Bronx|    6|    other|    23045|
|     Bronx|    7|     long|    84109|
|     Bronx|    7|    short|   237801|
|     Bronx|    7|    other|    23610|
|     Bronx|    8|    other|    23041|
|     Bronx|    8|    sho

o. Diferencias por vendor en avg_speed_mph y trip_duration_min.

In [15]:
vendor_stats = (
    df.groupBy("VENDOR_NAME")
      .agg(
          F.round(F.avg("AVG_SPEED_MPH"),2).alias("avg_speed"),
          F.round(F.avg("TRIP_DURATION_MIN"),2).alias("avg_duration")
      )
)
vendor_stats.show()

+--------------------+---------+------------+
|         VENDOR_NAME|avg_speed|avg_duration|
+--------------------+---------+------------+
|  Curb Mobility, LLC|    11.92|       21.32|
|Creative Mobile T...|    37.15|       16.31|
|       Not specified|    10.64|       14.17|
|               Helix|     NULL|        0.00|
+--------------------+---------+------------+



p. Relación método de pago ↔ tip_amount por hora.

In [16]:
tip_hour = (
    df.groupBy("PAYMENT_TYPE_DESC", "PICKUP_HOUR")
      .agg(F.round(F.avg("TIP_AMOUNT"),2).alias("avg_tip"))
      .orderBy("PICKUP_HOUR")
)
tip_hour.show(7*24)

+-----------------+-----------+-------+
|PAYMENT_TYPE_DESC|PICKUP_HOUR|avg_tip|
+-----------------+-----------+-------+
|          Dispute|          0|   0.01|
|      Credit card|          0|   3.49|
|             Cash|          0|    0.0|
|        No charge|          0|    0.0|
|          Unknown|          0|    0.0|
|      Credit card|          1|   3.35|
|        No charge|          1|    0.0|
|          Dispute|          1|   0.01|
|             Cash|          1|    0.0|
|          Unknown|          1|    0.0|
|          Dispute|          2|   0.01|
|        No charge|          2|    0.0|
|          Unknown|          2|    0.0|
|             Cash|          2|    0.0|
|      Credit card|          2|   2.84|
|        No charge|          3|   0.01|
|             Cash|          3|    0.0|
|      Credit card|          3|   2.71|
|          Dispute|          3|   0.01|
|          Unknown|          3|   0.03|
|          Dispute|          4|   0.01|
|      Credit card|          4|   2.74|


q. Zonas con percentil 99 de duración/distancia fuera de rango (posible congestión/eventos).

In [17]:
df_valid = df.filter(
    (F.col("TRIP_DISTANCE") > 0) &
    (F.col("TRIP_DURATION_MIN") > 0)
)

df_ratio = df_valid.withColumn(
    "duration_per_mile", 
    F.col("TRIP_DURATION_MIN") / F.col("TRIP_DISTANCE")
)

p99_duration_per_mile = df_ratio.approxQuantile("duration_per_mile", [0.99], 0.01)[0]

print(f"Percentil 99 de duración por milla: {p99_duration_per_mile:.2f} min/milla")

outliers_congestion = df_ratio.filter(
    F.col("duration_per_mile") >= p99_duration_per_mile
)

# VALORES IMPOSIBLES
outliers_physic = df_ratio.filter(
    (F.col("TRIP_DISTANCE") > 100) |
    (F.col("TRIP_DURATION_MIN") > 300)
)

outliers_total = outliers_congestion.union(outliers_physic)

zonas_congestion = (
    outliers_total.groupBy("PU_ZONE")
        .agg(F.count("*").alias("num_outlier_trips"))
        .orderBy(F.col("num_outlier_trips").desc())
)

zonas_congestion.limit(20).show(truncate=False)

Percentil 99 de duración por milla: 125373065.00 min/milla
+----------------------------+-----------------+
|PU_ZONE                     |num_outlier_trips|
+----------------------------+-----------------+
|JFK Airport                 |58456            |
|Times Sq/Theatre District   |42986            |
|Midtown Center              |42935            |
|LaGuardia Airport           |41769            |
|Penn Station/Madison Sq West|39465            |
|East Village                |36559            |
|Upper East Side South       |36067            |
|Clinton East                |35827            |
|Midtown East                |34229            |
|Upper East Side North       |32868            |
|Union Sq                    |32500            |
|Murray Hill                 |31892            |
|Midtown North               |30451            |
|Midtown South               |28526            |
|Lincoln Square East         |28488            |
|East Chelsea                |27992            |
|Upper Wes

r. Yield por milla (total_amount/trip_distance) por borough y hora.

In [18]:
# VALORES RAZONABLES
df_filtered = df.filter((F.col("TRIP_DISTANCE") > 0) & (F.col("TRIP_DISTANCE") < 1000))

df_yield = df_filtered.withColumn("yield", F.col("TOTAL_AMOUNT") / F.col("TRIP_DISTANCE"))


yield_stats = (
    df_yield.groupBy("PU_BOROUGH", "PICKUP_HOUR")
            .agg(F.round(F.avg("yield"), 2).alias("avg_yield"))

)

yield_stats.show()

+-------------+-----------+---------+
|   PU_BOROUGH|PICKUP_HOUR|avg_yield|
+-------------+-----------+---------+
|          N/A|          7|   236.17|
|    Manhattan|         15|     10.5|
|       Queens|         14|    12.11|
|     Brooklyn|         21|     8.57|
|          N/A|         21|   257.47|
|       Queens|          2|     13.9|
|          N/A|          9|   206.85|
|      Unknown|         15|    25.56|
|       Queens|          4|    11.03|
|     Brooklyn|         13|     10.4|
|    Manhattan|         17|     9.47|
|    Manhattan|          8|    10.92|
|       Queens|          8|    12.35|
|Staten Island|         21|     50.0|
|       Queens|          6|    10.31|
|       Queens|          7|    11.28|
|          N/A|         15|   320.32|
|          N/A|         11|   233.91|
|          EWR|         21|   745.54|
|     Brooklyn|         18|     8.77|
+-------------+-----------+---------+
only showing top 20 rows



s. Cambios YoY en volumen y ticket promedio por service_type.

In [19]:
# years solo pueden ser 2015 a 2025

df = df.filter((F.col("YEAR") >= 2015) & (F.col("YEAR") <= 2025))
# total amount debe tener sentido
df = df.filter((F.col("TOTAL_AMOUNT") > 0) & (F.col("TOTAL_AMOUNT") < 500))

yoy_service = (
    df.groupBy("SERVICE_TYPE", "YEAR")
      .agg(
          F.count("*").alias("num_trips"),
          F.round(F.avg("TOTAL_AMOUNT"),2).alias("avg_ticket")
      )
      .orderBy("SERVICE_TYPE", "YEAR")
)
yoy_service.show(100)

+------------+----+---------+----------+
|SERVICE_TYPE|YEAR|num_trips|avg_ticket|
+------------+----+---------+----------+
|       green|2015| 20613863|     14.81|
|       green|2016| 16294527|     14.72|
|       green|2017| 11673937|     14.31|
|       green|2018|  8743329|     15.83|
|       green|2019|  5582043|     16.09|
|       green|2020|  1191187|     15.84|
|       green|2021|   650470|     19.03|
|       green|2022|   742875|     18.44|
|       green|2023|   722192|     23.42|
|       green|2024|   626357|     24.07|
|       green|2025|   321781|     24.15|
|      yellow|2015|145926798|     16.03|
|      yellow|2016|132176283|     16.45|
|      yellow|2017| 75915278|     16.18|
|      yellow|2018|101781114|     16.38|
|      yellow|2019| 82434360|     19.08|
|      yellow|2020| 23197048|     17.89|
|      yellow|2021| 16354850|     18.36|
|      yellow|2022| 37266273|     21.69|
|      yellow|2023| 36036797|      29.0|
|      yellow|2024| 36063754|     29.13|
|      yellow|20

t. Días con alta congestion_surcharge: efecto en total_amount vs días “normales”

In [20]:
df_cong = df.withColumn("high_congestion", F.when(F.col("CONGESTION_SURCHARGE")>2, "high").otherwise("normal"))

congestion_effect = (
    df_cong.groupBy("PU_BOROUGH", "high_congestion")
            .agg(
                F.count("*").alias("num_trips"),
                F.round(F.avg("TOTAL_AMOUNT"),2).alias("avg_total_amount")
            )
)
congestion_effect.show()

+-------------+---------------+---------+----------------+
|   PU_BOROUGH|high_congestion|num_trips|avg_total_amount|
+-------------+---------------+---------+----------------+
|     Brooklyn|         normal| 31901103|           17.02|
|      Unknown|           high|  1159141|           25.69|
|     Brooklyn|           high|   762645|           31.02|
|Staten Island|         normal|    43446|           49.47|
|       Queens|           high| 10185300|           70.28|
|          EWR|         normal|    66104|           94.43|
|    Manhattan|           high|214547118|           19.56|
|        Bronx|           high|    31565|           43.92|
|Staten Island|           high|      705|           71.81|
|       Queens|         normal| 56594076|           34.83|
|    Manhattan|         normal|447448506|            14.5|
|      Unknown|         normal|  7673123|           18.22|
|          N/A|         normal|   422456|           79.35|
|          N/A|           high|     7667|           54.5